In [1]:
import math
import torch
import numpy as np
import gpytorch
import pandas as pd
from matplotlib import pyplot as plt
import random
from scipy.stats import norm
from scipy.optimize import minimize

In [2]:
filename = r'../data/olhs_run1.xlsx'
x_pd = pd.read_excel(filename, sheet_name='Initial Design (OLHS)', header=[0,1], index_col=[0])
y_pd = pd.read_excel(filename, sheet_name='bo_data', header=[0,1], index_col=[0])

In [3]:
y_pd

Group ID,Polymer Solubility,Gelation Enthalpy,Shear Modulus,Manufacturability
#,mg/mL,J/g,Kpa,--
1,51.344743,1.25180,4.965500,1
2,20.121238,56.63540,0.536800,0
3,35.672397,3.46590,1.036200,0
4,29.114323,2.56890,0.998800,0
5,123.417722,0.14074,1.899265,1
6,81.196581,88.70100,1.078000,1
7,84.919473,0.30367,2.343650,1
8,64.625850,27.80800,3.263690,1
9,47.923323,116.06000,5.088030,1


In [4]:
#normalizing the inputs
xmeans = x_pd.mean(axis=0)
xstddv = x_pd.std(axis=0)
x_pd_normal = (x_pd - xmeans)/xstddv

x_pd = x_pd_normal

x_mins = x_pd.min(axis=0).to_numpy()
x_maxs = x_pd.max(axis=0).to_numpy()

In [5]:
#normalize objective value data
y_obj_pd = y_pd.iloc[:, [0,1,2]]
ymeans = y_obj_pd.mean(axis=0)
ystddv = y_obj_pd.std(axis=0)
y_obj_pd_normal = (y_obj_pd - ymeans) / ystddv

y_pd.iloc[:, [0,1,2]] = y_obj_pd_normal

In [6]:
validation_idx = [1,7,15]

train_x_pd = x_pd.drop(validation_idx)
train_y_pd = y_pd.drop(validation_idx)

In [7]:
#make torch tensors
train_x = torch.tensor(train_x_pd.values, dtype=torch.float)
train_y1 = torch.tensor(train_y_pd['Polymer Solubility', 'mg/mL'].values, dtype=torch.float).squeeze()
train_y2 = torch.tensor(train_y_pd['Gelation Enthalpy', 'J/g'].values, dtype=torch.float).squeeze()
train_y3 = torch.tensor(train_y_pd['Shear Modulus', 'Kpa'].values, dtype=torch.float).squeeze()
train_y4 = torch.tensor(train_y_pd['Manufacturability', '--'].values, dtype=torch.long).squeeze()

In [8]:
test_x = torch.tensor(x_pd.values, dtype=torch.float)
test_y = torch.tensor(y_pd.values, dtype=torch.float).squeeze()

In [9]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    
# function to optimize parameters of the regression GP -
def train_reg_gp(model, likelihood, train_x, train_y, training_iter):
   # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for i in range(training_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(train_x)
        # Calc loss and backprop gradients
        loss = -mll(output, train_y)
        loss.backward()
        if i - 1  == training_iter:
            print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
                i + 1, training_iter, loss.item(),
                model.covar_module.base_kernel.lengthscale.item(),
                model.likelihood.noise.item()
            ))
        optimizer.step()

    return model, likelihood 

In [10]:
class DirichletGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, num_classes):
        super(DirichletGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean(batch_shape=torch.Size((num_classes,)))
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(batch_shape=torch.Size((num_classes,))),
            batch_shape=torch.Size((num_classes,)),
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# function to optimize parameters of the classification GP - 
def train_cls_gp(model, likelihood, train_x, training_iter):
   # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for i in range(training_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(train_x)
        # Calc loss and backprop gradients
        loss = -mll(output, likelihood.transformed_targets).sum()
        loss.backward()
        optimizer.step()

    return model, likelihood


In [11]:
#define likelihood and initialize model - regression
reg_likl = gpytorch.likelihoods.GaussianLikelihood()
reg_model1 = ExactGPModel(train_x, train_y1, reg_likl)
reg_model2 = ExactGPModel(train_x, train_y2, reg_likl)
reg_model3 = ExactGPModel(train_x, train_y3, reg_likl)

#define likelihood and initialize model - classification
cls_likl = gpytorch.likelihoods.DirichletClassificationLikelihood(train_y4, learn_additional_noise=False, alpha_epsilon=1e-4)
cls_model = DirichletGPModel(train_x, cls_likl.transformed_targets, cls_likl, num_classes=cls_likl.num_classes)

In [12]:
training_iter = 50

#train model - regression
reg_model1, reg_likl1 = train_reg_gp(reg_model1, reg_likl, train_x, train_y1, training_iter)
reg_model2, reg_likl2 = train_reg_gp(reg_model2, reg_likl, train_x, train_y2, training_iter)
reg_model3, reg_likl3 = train_reg_gp(reg_model3, reg_likl, train_x, train_y3, training_iter)

#train model - regression
cls_model, cls_likl = train_cls_gp(cls_model, cls_likl, train_x, training_iter)

In [40]:
def predict(model, likl, X):
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        pred = likl(model(X))
        mean = pred.mean
        std = pred.stddev
    return mean, std

#sum the available models to convert mutli obj problem to single obj
def multi_objective_predict(X, models, liklihoods):
    mean = 0
    var = 0
    for i in range(len(models)):
        tmp_mean, tmp_std = predict(models[i], liklihoods[i], torch.tensor(X, dtype=torch.float))
        mean += tmp_mean
        var += tmp_std**2
    return mean, var**0.5

# function to calculate acquisition function 
def acquisition_func(X, models, liklihoods, y_min_curr):
    X = np.reshape(X, (1, -1))
    pred_mean, pred_std = multi_objective_predict(X, models, liklihoods) 
    improv = pred_mean - y_min_curr
    z_score = np.divide(improv, pred_std + 1E-9)
    acf = np.multiply(improv, norm.cdf(z_score)) + np.multiply(pred_std, norm.pdf(z_score))
    return (-1.0) * acf 

In [55]:
# search for best point
n_restarts = 100
# x_bounds = np.array([[2000, 10000], [0, 100], [0, 40], [5000, 15000], [80, 100], [0,100], [60, 100], [70, 100]])
x_bounds = np.c_[x_mins, x_maxs]
search_grid = np.random.uniform(x_bounds[:, 0], x_bounds[:, 1], size=(10*n_restarts, len(x_bounds)))

mo_models = [reg_model1, reg_model2]
mo_likls = [reg_likl1, reg_likl2]

# Swith on eval mode
for i in range(len(mo_models)):
    mo_models[i].eval()
    mo_likls[i].eval()

y_best_curr = torch.max((train_y1 + train_y2)).item()

acf_vals = [acquisition_func(search_grid[i, :].reshape(1, -1), mo_models, mo_likls, y_best_curr) for i in range(10*n_restarts)]

acf_vals = np.array(acf_vals).reshape(10*n_restarts,)
top_idx = np.argsort(acf_vals)
search_grid = search_grid[top_idx[0:n_restarts]].reshape(n_restarts,-1)

best_acquisition_values = 1
best_x = None
for i, starting_point in enumerate(search_grid):
    res = minimize(fun=acquisition_func, 
                   x0=starting_point, 
                   bounds=x_bounds,
                   method='L-BFGS-B',
                   args=(mo_models, mo_likls, y_best_curr))
    if res.fun < best_acquisition_values:
        best_acquisition_values = res.fun
        best_x = res.x

new_point = best_x
new_acf = (-1.0) * best_acquisition_values

In [56]:
# inverse transform 
def inverse_transform(val, mean, std):
    return std * val + mean

In [57]:
pred, std = multi_objective_predict(new_point.reshape(1,-1), mo_models, mo_likls)
print('Predicted output at new point - ', pred)
print('std deviation - ', std)

Predicted output at new point -  tensor([2.4799])
std deviation -  tensor([1.1549])


In [58]:
new_pt_tensor = torch.tensor(new_point.reshape(1,-1), dtype=torch.float)
y1_pred, _ = predict(reg_model1, reg_likl1, new_pt_tensor)
y2_pred, _ = predict(reg_model2, reg_likl2, new_pt_tensor)
print(y1_pred, ' and ', y2_pred)

tensor([2.4151])  and  tensor([0.0647])


In [59]:
new_point

array([ 1.22111523, -1.14006761, -0.46216723,  1.24853254,  0.43425369,
        0.17214098, -1.4405752 ,  1.41954212])

In [60]:
y1_pred = inverse_transform(y1_pred, ymeans.values[0], ystddv.values[0])
y2_pred = inverse_transform(y2_pred, ymeans.values[1], ystddv.values[1])
print(y1_pred)

tensor([179.7163])


In [61]:
inverse_transform(y_best_curr, ymeans.values[0], ystddv.values[0])

143.9980154794394

In [67]:
new_pt_tr = [inverse_transform(new_point[i], xmeans.values[i], xstddv.values[i]) for i in range(len(new_point))]
new_pt_tr

[9041.774785340262,
 14.50141610594094,
 14.243751126584717,
 13887.588474175576,
 92.70429419988953,
 55.35999886869985,
 62.05777298547319,
 98.26019646051431]